In [4]:
# Import Libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
import random
import warnings
warnings.simplefilter('ignore')

from matplotlib.pyplot import imshow
from keras.preprocessing import image
from keras import applications
import os
import glob

import keras
from keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input,Dense,Activation,ZeroPadding2D,BatchNormalization,Flatten,Conv2D
from keras.layers import AveragePooling2D,GlobalAveragePooling2D,GlobalMaxPool2D,MaxPooling2D,MaxPool2D,Dropout
from keras.models import Model,Sequential

In [16]:
data_dir = '/Users/Zeeshan/Documents/UofT ML Course/Project/uoft-ml-sports/data'

In [ ]:
train = glob.glob(data_dir + '/train/*')
train_class=os.listdir(data_dir + '/train')


['lacrosse',
 'shuffleboard',
 'arm wrestling',
 'golf',
 'surfing',
 'snow boarding',
 'parallel bar',
 'pole climbing',
 'giant slalom',
 'bobsled',
 'baton twirling',
 'weightlifting',
 'hydroplane racing',
 'barell racing',
 'water cycling',
 'mushing',
 'uneven bars',
 'pole vault',
 'roller derby',
 'rowing',
 'jousting',
 'air hockey',
 'sky surfing',
 'shot put',
 'sidecar racing',
 'field hockey',
 'harness racing',
 'disc golf',
 'canoe slamon',
 'football',
 'ski jumping',
 'ice yachting',
 'cricket',
 'figure skating women',
 'horse racing',
 'baseball',
 'billiards',
 'bowling',
 'motorcycle racing',
 'javelin',
 'fly fishing',
 'bull riding',
 'sumo wrestling',
 'tug of war',
 'figure skating men',
 'sailboat racing',
 'horse jumping',
 'basketball',
 'cheerleading',
 'frisbee',
 'high jump',
 'water polo',
 'polo',
 'rock climbing',
 'rollerblade racing',
 'figure skating pairs',
 'balance beam',
 'formula 1 racing',
 'volleyball',
 'hang gliding',
 'ultimate',
 'croquet

In [22]:
count_dict1 = {}
img_dict1 = {}

# Loop through classes
for cls in train_class:  # Assuming class_names contains the list of dog classes
    image_path = glob.glob(data_dir + f'/train/{cls}/*')
    count_dict1[cls] = len(image_path)

    if image_path:  # Check if image_path is not empty
        img_dict1[cls] = tf.keras.utils.load_img(random.choice(image_path))
count_dict1

{'lacrosse': 151,
 'shuffleboard': 130,
 'arm wrestling': 99,
 'golf': 157,
 'surfing': 142,
 'snow boarding': 117,
 'parallel bar': 131,
 'pole climbing': 118,
 'giant slalom': 150,
 'bobsled': 138,
 'baton twirling': 108,
 'weightlifting': 134,
 'hydroplane racing': 108,
 'barell racing': 123,
 'water cycling': 103,
 'mushing': 111,
 'uneven bars': 156,
 'pole vault': 138,
 'roller derby': 163,
 'rowing': 143,
 'jousting': 118,
 'air hockey': 112,
 'sky surfing': 59,
 'shot put': 149,
 'sidecar racing': 133,
 'field hockey': 157,
 'harness racing': 147,
 'disc golf': 123,
 'canoe slamon': 164,
 'football': 191,
 'ski jumping': 130,
 'ice yachting': 112,
 'cricket': 129,
 'figure skating women': 157,
 'horse racing': 139,
 'baseball': 174,
 'billiards': 145,
 'bowling': 120,
 'motorcycle racing': 132,
 'javelin': 136,
 'fly fishing': 134,
 'bull riding': 149,
 'sumo wrestling': 133,
 'tug of war': 128,
 'figure skating men': 128,
 'sailboat racing': 144,
 'horse jumping': 134,
 'baske

In [23]:
df1 = pd.DataFrame(data={'label':count_dict1.keys(),'count':count_dict1.values()})
df1

,label,count
0,lacrosse,151
1,shuffleboard,130
2,arm wrestling,99
3,golf,157
4,surfing,142
...,...,...
95,track bicycle,135
96,chuckwagon racing,120
97,hurdles,136
98,horseshoe pitching,102


In [28]:
# Data Preprocessing

train_data = tf.keras.utils.image_dataset_from_directory(data_dir + '/train',label_mode='categorical',shuffle=False)
test_data = tf.keras.utils.image_dataset_from_directory(data_dir + '/test',shuffle=False,label_mode='categorical')
validation_data = tf.keras.utils.image_dataset_from_directory(data_dir + '/valid',label_mode='categorical',shuffle=False)

Found 13492 files belonging to 100 classes.
Found 500 files belonging to 100 classes.
Found 500 files belonging to 100 classes.


In [48]:
width = 229
height = 229
channels = 3

data_preprocessing = tf.keras.Sequential([
    tf.keras.layers.Resizing(height, width),
    tf.keras.layers.Rescaling(1.0 / 255),
])

In [49]:
train_ds = train_data.map(lambda x,y:(data_preprocessing(x),y))
test_ds = test_data.map(lambda x,y:(data_preprocessing(x),y))
valid_ds = validation_data.map(lambda x,y:(data_preprocessing(x),y))

In [56]:
# Data Augmentation

batch_size = 40
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
  shear_range=0.2,
  zoom_range=0.2,
  horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
  data_dir + '/train',  # this is the target directory
  target_size=(229, 229),  # all images will be resized 
  batch_size=batch_size,
  class_mode='categorical',
  shuffle = True,  seed = 42)  

val_datagen = ImageDataGenerator()

validation_generator = val_datagen.flow_from_directory(
  data_dir + '/valid',
  target_size=(229, 229),
  batch_size=batch_size,
  class_mode='categorical',
  shuffle = False,seed = 42 )

Found 13492 images belonging to 100 classes.
Found 500 images belonging to 100 classes.


In [57]:
batch_images, batch_labels = next(train_generator)

# Print the shape of the batch
print("Batch images shape:", batch_images.shape)
print("Batch labels shape:", batch_labels.shape)

Batch images shape: (40, 229, 229, 3)
Batch labels shape: (40, 100)


In [52]:
# RESNET50 Model

from tensorflow.keras.applications import EfficientNetB0
# Load the EfficientNetB0 model pre-trained on ImageNet
base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(229, 229, 3))

# Freeze the base model
for layer in base_model.layers:
    layer.trainable = False
    
# Create a new model
model = Sequential()

# Add the pre-trained VGG16 base model
model.add(base_model)
model.add(GlobalAveragePooling2D())
# Add a dense layer with 256 neurons and ReLU activation
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.25))

# Add a dense layer with the number of output classes and softmax activation
model.add(Dense(100, activation='softmax'))

In [53]:
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [54]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=1e-7)

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy','Precision','Recall'])

history = model.fit(
  train_generator,
  epochs=10,
  validation_data=validation_generator,
  callbacks=[early_stopping, reduce_lr])

Epoch 1/10
338/338 ━━━━━━━━━━━━━━━━━━━━ 607s 2s/step - Precision: 0.8224 - Recall: 0.2397 - accuracy: 0.4578 - loss: 2.3917 - val_Precision: 0.9675 - val_Recall: 0.8920 - val_accuracy: 0.9360 - val_loss: 0.2701 - learning_rate: 0.0010
Epoch 2/10
338/338 ━━━━━━━━━━━━━━━━━━━━ 589s 2s/step - Precision: 0.9210 - Recall: 0.7831 - accuracy: 0.8526 - loss: 0.5151 - val_Precision: 0.9751 - val_Recall: 0.9400 - val_accuracy: 0.9560 - val_loss: 0.1543 - learning_rate: 0.0010
Epoch 3/10
338/338 ━━━━━━━━━━━━━━━━━━━━ 609s 2s/step - Precision: 0.9403 - Recall: 0.8547 - accuracy: 0.9001 - loss: 0.3492 - val_Precision: 0.9691 - val_Recall: 0.9400 - val_accuracy: 0.9540 - val_loss: 0.1576 - learning_rate: 0.0010
Epoch 4/10
 17/338 ━━━━━━━━━━━━━━━━━━━━ 9:21 2s/step - Precision: 0.9549 - Recall: 0.8782 - accuracy: 0.9251 - loss: 0.2560